In [1]:
from pytezos import ContractInterface, pytezos, MichelsonRuntimeError
from decimal import Decimal
import time
from settings import settings

run_time = int(time.time())
one_hour = 60*60

In [2]:
contract_fn = 'crystal_ball_turbo.tz'
participants = dict(
    a='tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ',
    b='tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos',
    c='tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE'
)
contract = ContractInterface.from_file(contract_fn)

In [3]:
storage = {
    'currencyPair': 'XTZ-USD',
    'createdTime': run_time,
    'targetDynamics': 1_000_000,
    'betsCloseTime': run_time + 24*one_hour,
    'measuseStartTime': 0,
    'measureOracleStartTime': 0,
    'isMeasurementStarted': False,
    'startRate': 0,
    'measurePeriod': 12*one_hour,
    'isClosed': False,
    'closedTime': 0,
    'closedOracleTime': 0,
    'closedRate': 0,
    'closedDynamics': 0,
    'isBetsForWin': False,

    'betsForLedger': {},
    'betsAgainstLedger': {},
    'liquidityLedger': {},

    'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',

    'betsForSum': 0,
    'betsAgainstSum': 0,
    'liquiditySum': 0,

    'liquidityPercent': 0,
    'expirationFee': 0
}

In [4]:
# participant A: adding liquidity 50/50:
res = contract.bet(betAgainst=50_000, betFor=50_000).with_amount(100_000).interpret(
    storage=storage, sender=participants['a'], now=run_time)

In [5]:
assert res.storage['betsForSum'] == 50_000
assert res.storage['betsAgainstSum'] == 50_000
assert len(res.storage['betsForLedger']) == 1
assert len(res.storage['betsAgainstLedger']) == 1
assert len(res.storage['liquidityLedger']) == 1
assert res.storage['liquidityLedger'][participants['a']] == 50_000

In [6]:
# TODO: check assertRaises contract.bet(betAgainst=0, betFor=50_000).with_amount(100_000)

In [7]:
# participant B: bets for 10_000 after 1 hour
res = contract.bet(betAgainst=0, betFor=50_000).with_amount(50_000).interpret(
    storage=res.storage, sender=participants['b'], now=run_time + one_hour)

assert res.storage['betsForSum'] == 100_000
assert res.storage['betsAgainstSum'] == 50_000
assert len(res.storage['betsForLedger']) == 2
assert len(res.storage['betsAgainstLedger']) == 1
assert len(res.storage['liquidityLedger']) == 1

In [8]:
res.storage['liquidityLedger']

{'tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ': 50000}

In [9]:
# participant A: adding more liquidity after 12 hours (exactly half of the betting period):
res = contract.bet(betAgainst=100_000, betFor=50_000).with_amount(150_000).interpret(
    storage=res.storage, sender=participants['a'], now=run_time + 12*one_hour)

assert res.storage['betsForSum'] == 150_000
assert res.storage['betsAgainstSum'] == 150_000
assert len(res.storage['betsForLedger']) == 2
assert len(res.storage['betsAgainstLedger']) == 1
assert len(res.storage['liquidityLedger']) == 1
assert res.storage['liquiditySum'] == 50_000 + 25_000

In [10]:
res.storage['liquiditySum']

75000

In [11]:
# participant C: adding more liquidity at the very end:
res = contract.bet(betAgainst=500_000, betFor=500_000).with_amount(1_000_000).interpret(
    storage=res.storage, sender=participants['c'], now=run_time + 24*one_hour)

assert res.storage['betsForSum'] == 650_000
assert res.storage['betsAgainstSum'] == 650_000
assert len(res.storage['betsForLedger']) == 3
assert len(res.storage['betsAgainstLedger']) == 2
assert len(res.storage['liquidityLedger']) == 2
assert res.storage['liquiditySum'] == 50_000 + 25_000 + 0

In [12]:
assert res.storage['betsForSum'] == sum(res.storage['betsForLedger'].values())
assert res.storage['betsAgainstSum'] == sum(res.storage['betsAgainstLedger'].values())
assert res.storage['liquiditySum'] == sum(res.storage['liquidityLedger'].values())

In [13]:
# test running measurePeriod after 24h + 2h delay
# test closing 

In [14]:
res.storage

{'betsAgainstLedger': {'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE': 500000,
  'tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ': 150000},
 'betsAgainstSum': 650000,
 'betsCloseTime': 1618070125,
 'betsForLedger': {'tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos': 50000,
  'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE': 500000,
  'tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ': 100000},
 'betsForSum': 650000,
 'closedDynamics': 0,
 'closedOracleTime': 0,
 'closedRate': 0,
 'closedTime': 0,
 'createdTime': 1617983725,
 'currencyPair': 'XTZ-USD',
 'expirationFee': 0,
 'isBetsForWin': False,
 'isClosed': False,
 'isMeasurementStarted': False,
 'liquidityLedger': {'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE': 0,
  'tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ': 75000},
 'liquidityPercent': 0,
 'liquiditySum': 75000,
 'measureOracleStartTime': 0,
 'measurePeriod': 43200,
 'measuseStartTime': 0,
 'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',
 'startRate': 0,
 'targetDynamics': 1000000}

### assert что сделку нельзя сделать после закрытия времени

### TODO: случайный тест
- генерировать много разных участников
- генерировать случайные действия, накапливать сумму, потом сравнивать что суммы правильные (по реестрам)
- в идеале даже полностью прогонять какой-то ивент:
    - прибавлять время по k минут
    - генерировать случайные события
    - потом кто-то закрывает контракт
    - потом выводят средства, суммируются выходы с транзакций, проверяется что всё сходится
- генерировать разное время и проверять что ошибка если время не в интервале
- запускать этот случайный тест несколько раз
- лучше несколько простых случайных тестов, каждый из которых тестит какую-то часть
- хотя можно один полный с различными вариантами ставок


### TODO: move test to unittest!

### TODO: make script to compile and run tests from tests folder!